In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
data_path = "./dane/"
df = pd.read_csv(os.path.join(data_path, "szer.csv"), sep=';')
df = df.drop("KR", axis=1)

In [4]:
# wartość optymalizacji po poszczególnym ruchu
optimalization = pd.DataFrame(columns=["Wartosc kombinacji"])
Neighbours_count = 20
improvements = np.zeros(Neighbours_count)
np.random.seed(42)


In [22]:
# KR to wartość naszego rozwiązania - ma być najmniejsza
def E(df = df):
    c_sum = df["Czas wykonania"].cumsum() # suma kumultatywna kolumny "Czas wykonania"
    KR = (df["Termin"] - c_sum)**2 # obliczanie kwadratu różnicy wartosći
    return sum(KR) # KR jest wskaźnikiem jakości ułożenia zadań. To optymalizujemy

# wyszukiwanie listy sąsiadów
def get_neighbours(idx, neighbours_count = 20, df = df):
    N = neighbours_count
    # Idea sąsiedztwa dla N = 6 i idx = n E <N/2 ; L-N/2>
    # idx = X, sąsiad = 0, nie sąsiad = -
    # nr_indeksu | ... | n-4 | n-3 | n-2 | n-1 |  n  | n+1 | n+2 | n+3 | n+4 | ... |
    # sasiedztwo |  -  |  -  |  0  |  0  |  0  |  X  |  0  |  0  |  0  |  -  |  -  |
    
    L = len(df) - 1 
    idx_prev = idx # indeks dla ktorego szukamy sasiadow
    N_half = N//2 # polowa wartosci N
    
    # Idea sąsiedztwa dla N = 6 i idx = n < N/2
    # idx = X, sąsiad = 0, nie sąsiad = -
    # przykład n = 2
    # nr_indeksu |  0  |  1  | n=2 |  3  |  4  |  5  |  6  |  7  |  8  |  9  | ... |
    # sasiedztwo |  0  |  0  |  X  |  0  |  0  |  0  |  0  |  -  |  -  |  -  |  -  |
    # skoro nie mozemy znaleźć wystarczająco dużo sąsiadów po lewej, 
    # szukamy dodatkowego po prawej
    if idx < N_half:
        idx = N_half
    # idea sąsiedztwa z ograniczeniem z prawej jest analogiczna do tej z ograniczeniem z lewej
    elif idx > L - N_half:
        idx = L - N_half
    output = list(range(idx - N_half, idx + N_half + 1))
    # usuwanie idx z output
    # TODO: na pewno da się to zrobić prościej
    rm_idx = output.index(idx_prev)
    output = np.array(output)
    output = np.delete(output, rm_idx)
    return output

def reduce_temperature(T, rate_of_change = .02, sposob = 1):
    return (1-stopien_zmiany)*T if sposob == 1 else T/(1+stopien_zmiany*T)



In [23]:
R = 42 # rozwiązanie
T = 1000 # jakaś losowa liczba (?)
RANDOM = np.random.rand() # TODO: jaki zakres tutaj dać?
while T > 100:
    break_counter = 10
    while break_counter:
        neighbour = get_neighbours(idx=R, neighbours_count=20)
        R_prim = np.random.choice(neighbours) # losowy indeks wiersza
        dE = E(R_prim) - E(R) # TODO
        if dE < 0 or RANDOM < np.exp(-dE/T):
            R = R_prim
        else:
            break_counter -= 1
    T = reduce_temperature(T)
            

ValueError: a must be 1-dimensional or an integer